# Analysis Set up

This Notebook will hold some gerneral analysis for data concerning US car advertisements. The analysis will focus more on the day listing of the cars which will be transfered to the web app.

In [41]:
import pandas as pd   #Importing packages
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import streamlit as st

`Data Upload and duplicate/blank value fix`

In [42]:
#upload vehicle csv reading the top lines
vehicles = pd.read_csv(r'C:\Users\treve\sprint5project\vehicles_us.csv')
vehicles.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [43]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [44]:
vehicles.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

Change NaN values in `is_4wd` column to 0 

In [45]:
print(vehicles['is_4wd'].value_counts(dropna=False))
vehicles['is_4wd'] = vehicles['is_4wd'].fillna(0)
vehicles['is_4wd'].value_counts(dropna=False)



is_4wd
NaN    25953
1.0    25572
Name: count, dtype: int64


is_4wd
0.0    25953
1.0    25572
Name: count, dtype: int64

Checking for Duplicates

In [46]:
vehicles.duplicated().sum()  #No duplicates

0

# Create general aggregates and charts


Looking at general aggregates of various columns from the data set.

In [47]:
vehicles['price'].describe()

count     51525.000000
mean      12132.464920
std       10040.803015
min           1.000000
25%        5000.000000
50%        9000.000000
75%       16839.000000
max      375000.000000
Name: price, dtype: float64

In [48]:
vehicles['days_listed'].describe()

count    51525.00000
mean        39.55476
std         28.20427
min          0.00000
25%         19.00000
50%         33.00000
75%         53.00000
max        271.00000
Name: days_listed, dtype: float64

`look at type data`

In [49]:
top_types = vehicles['type'].value_counts().head(10)
top_types

type
SUV            12405
truck          12353
sedan          12154
pickup          6988
coupe           2303
wagon           1541
mini-van        1161
hatchback       1047
van              633
convertible      446
Name: count, dtype: int64

In [93]:
#Change pickup to truck as they are essentially the same type and creat top type counts
vehicles['type'] = vehicles['type'].replace('pickup','truck')
type_counts = vehicles['type'].value_counts().nlargest(5).index
type_counts

Index(['truck', 'SUV', 'sedan', 'coupe', 'wagon'], dtype='object', name='type')

In [94]:
#filter by top counts for type for histogram and scatter plot
filtered_types = vehicles[vehicles["type"].isin(type_counts)]
filtered_types.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,car_make,car_model
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw,x5
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,truck,white,1.0,2018-10-19,50,ford,f-150
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,0.0,2019-02-07,79,hyundai,sonata
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,truck,NaN,0.0,2019-03-22,9,ford,f-150
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,0.0,2019-04-02,28,chrysler,200


In [74]:
px.histogram(filtered_types,x='days_listed',title='Days Listed Histogram',template='plotly',color='type').update_yaxes(title_text = 'frequency of listing length').update_xaxes(title_text = "days listed")


The above histogram shows that the top car types will be listed for around 20 days and that most cars will rarely break 100 days listing. Truck and SUV type vehicles are the most popular.

In [75]:
px.scatter(filtered_types,x='price',y='days_listed',title='Price vs Days Listed Scatter',template='plotly',color='type').update_yaxes(title_text = "days listed")

From looking at the scatter plot wagons are the most concentrated while also being list relatively shortly and cheaply compared to other car types.

`Look at paint color data`

In [54]:
print(vehicles['paint_color'].value_counts())
top_paint_counts = vehicles['paint_color'].value_counts().head()

paint_color
white     10029
black      7692
silver     6244
grey       5037
blue       4475
red        4421
green      1396
brown      1223
custom     1153
yellow      255
orange      231
purple      102
Name: count, dtype: int64


`Car Makes`

In [55]:
vehicles['model']

0                bmw x5
1            ford f-150
2        hyundai sonata
3            ford f-150
4          chrysler 200
              ...      
51520     nissan maxima
51521       honda civic
51522    hyundai sonata
51523    toyota corolla
51524     nissan altima
Name: model, Length: 51525, dtype: object

Going to want to split the `model` column into two columns that designate the make and model of the car

In [82]:
vehicles[['car_make', 'car_model']] = vehicles['model'].str.split(' ', n=1, expand=True)
vehicles[['model','car_make','car_model']]

,model,car_make,car_model
0,bmw x5,bmw,x5
1,ford f-150,ford,f-150
2,hyundai sonata,hyundai,sonata
3,ford f-150,ford,f-150
4,chrysler 200,chrysler,200
...,...,...,...
51520,nissan maxima,nissan,maxima
51521,honda civic,honda,civic
51522,hyundai sonata,hyundai,sonata
51523,toyota corolla,toyota,corolla


In [58]:
make_counts = vehicles['car_make'].value_counts()
make_counts


car_make
ford             12672
chevrolet        10611
toyota            5445
honda             3485
ram               3316
jeep              3281
nissan            3208
gmc               2378
subaru            1272
dodge             1255
hyundai           1173
volkswagen         869
chrysler           838
kia                585
cadillac           322
buick              271
bmw                267
acura              236
mercedes-benz       41
Name: count, dtype: int64

Filter the dataset so that it filters out the cars that are not made by the most popular car manufactures up for sale.

In [88]:
#top car makes
make_counts = vehicles['car_make'].value_counts().nlargest(5).index
make_counts


Index(['ford', 'chevrolet', 'toyota', 'honda', 'ram'], dtype='object', name='car_make')

In [92]:
#filtered data set
filtered_makes = vehicles[vehicles["car_make"].isin(make_counts)]
filtered_makes

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,car_make,car_model
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,truck,white,1.0,2018-10-19,50,ford,f-150
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,truck,NaN,0.0,2019-03-22,9,ford,f-150
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,0.0,2018-12-27,73,toyota,camry
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68,honda,pilot
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17,honda,pilot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51515,12000,2005.0,chevrolet silverado 2500hd,good,8.0,diesel,228000.0,automatic,truck,silver,1.0,2018-08-18,52,chevrolet,silverado 2500hd
51518,3750,2005.0,ford taurus,excellent,6.0,gas,110200.0,automatic,sedan,silver,0.0,2018-08-10,63,ford,taurus
51519,11750,2015.0,honda accord,excellent,4.0,gas,NaN,automatic,coupe,NaN,0.0,2018-11-24,89,honda,accord
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,0.0,2018-11-14,22,honda,civic


In [90]:
px.histogram(filtered_makes,x='days_listed',title='Days Listed Histogram',template='plotly',color='car_make').update_yaxes(title_text = 'frequency of listing length').update_xaxes(title_text = "days listed")

From the chart above the cop car makes for sale only stay listed for the most part around 25 days with Chevy and Ford being the most popular.

In [91]:
px.scatter(filtered_makes,x='price',y='days_listed',title='Price vs Days Listed Scatter',template='plotly',color='car_make').update_yaxes(title_text = "days listed")

Most cars from the top car brands will be listed for under $50K and under 100 days. With Ram appearing to be most grouped together. All the while the most expensive cars do not necissarily stay advertised the longest.

# Conclusion

By looking the the histograms and scatter plots most cars will sell for under $50K and not be listed for longer for 150 days regardless of looking at top Car Types or Car Makes. Most cars will be listed for around 20 to 30 days and are highly likely to be a Chevy or Ford SUV/Truck.